# Retrieve, reformat, clean and store data of the Covid-19 pandemic

Data sources:
* John Hopkins University (**JHU**) - Center for System Science and Engineering (CSSE)
* Berliner Morgenpost (**BMP**)

In [ ]:
%reset -f
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from collections import namedtuple, OrderedDict
from pathlib import Path

plt.rcParams['figure.figsize'] = (15, 8)
pd.options.display.max_rows = 8
sns.set()

## Retrieving JHU data
* Dashboard: https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6
* Data: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data

Data is available as git repo, thus transfer is compressed. It is assumed that the `COVID-19` repo is `git clone`'d at the same directory level as this `covid-19_analysis`.
```
cd ..
git clone  https://github.com/CSSEGISandData/COVID-19.git
```

In [ ]:
covid_19_jhu_repo = Path('../COVID-19')

my_pwd = %pwd
%cd {covid_19_jhu_repo}
!git pull
%cd {my_pwd}

In [ ]:
filename_confirmed_global_JHU = (covid_19_jhu_repo /
  "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
assert filename_confirmed_global_JHU.exists()

confirmed_global_JHU = pd.read_csv(filename_confirmed_global_JHU)
print(confirmed_global_JHU.keys()[:5])
confirmed_global_JHU.set_index("Country/Region", inplace=True)
confirmed_global_JHU.head(3)

## Reformat JHU data

We are interested in the contries:
* Austria
* Germany
* France

We want the table have the data as coloumn (thus transpose it).

And the France data is scattered over the departments, thus we need to collect it.

In [ ]:
COUNTRIES_SELECTED = ["Germany", "Austria", "France"]

date_columns = [c for c in confirmed_global_JHU.columns if c.endswith('/20')]
cases_JHU = (confirmed_global_JHU
    .loc[confirmed_global_JHU.index.isin(COUNTRIES_SELECTED), date_columns]
    .transpose()
    .reset_index()
    .rename(columns={'index': 'date'})
)
cases_JHU['date'] = pd.to_datetime(cases_JHU['date'])
cases_JHU.set_index('date', inplace=True)
print(cases_JHU.tail(1))

# there are multiple France, let's sum them to one new France2
cases_JHU['France2'] = cases_JHU['France'].sum(axis=1)
del cases_JHU['France']
_rd = OrderedDict([
    ('Germany', 'Germany_JHU'),
    ('Austria', 'Austria_JHU'),
    ('France2', 'France_JHU'),
])
cases_JHU.rename(columns=_rd, inplace=True)
cases_JHU = cases_JHU.reindex(_rd.values(), axis=1)
cases_JHU.tail(4)

## Plotting JHU data

In [ ]:
#cases_JHU.plot(style='o-', logy=True)

## Retrieving BMP data

Total and recovered data are taken manually from the following URL on 2020-04-11:
* https://interaktiv.morgenpost.de/corona-virus-karte-infektionen-deutschland-weltweit/

In [ ]:
# print newest date from JHU to align
cases_JHU.tail(1)

In [ ]:
CasesBMP = namedtuple('CasesBMP', 'date Germany_confirmed_BMP Germany_recovered_BMP'.split())
D = lambda month, day: datetime(2020, month, day)
_cbmp = [
    # manually add new cases here
    #(D(4, 13), 127854, 60260),
    (D(4, 12), 127459, 60260), (D(4, 11), 124288, 58190), (D(4, 10), 120157, 55980),
    (D(4, 9), 115523, 52407),  (D(4, 8), 111779, 43656),  (D(4, 7), 107458, 37287),
    (D(4, 6), 103493, 33300),  (D(4, 5), 100024, 30600),  (D(4, 4), 95614, 28700),
    (D(4, 3), 91159, 26400),   (D(4, 2), 84788, 23800),   (D(4, 1), 77779, 21400),    
    (D(3, 31), 70985, 18700),  (D(3, 30), 66125, 16100),  (D(3, 29), 58655, 13500),
    (D(3, 28), 54268, 11500),  (D(3, 27), 49039, 9600),   (D(3, 26), 43211, 7600),
    (D(3, 25), 37613, 5900),   (D(3, 24), 32991, 5600),   (D(3, 23), 29056, 2753),
    (D(3, 22), 24875, 268),    (D(3, 21), 22366, 241),    (D(3, 20), 19850, 182),
    (D(3, 19), 15322, 117),    (D(3, 18), 12329, 107),    (D(3, 17), 9362, 73),
    (D(3, 16), 7274, 69),      (D(3, 15), 5815, 48),      (D(3, 14), 4587, 48),
    (D(3, 13), 3677, 48),      (D(3, 12), 2747, 27),      (D(3, 11), 1969, 27),
    (D(3, 10), 1567, 20),      (D(3, 9), 1146, 16),       (D(3, 8), 900, 16),
    (D(3, 7), 797, 16),        (D(3, 6), 641, 16),        (D(3, 5), 402, 16),
    (D(3, 4), 265, 16),        (D(3, 3), 190, 16),        (D(3, 2), 152, 16),
]    
cases_BMP = pd.DataFrame((CasesBMP(*c) for c in reversed(_cbmp))).set_index('date')
cases_BMP.tail(3)

## Plotting BMP data

In [ ]:
#cases_BMP.plot(style='o-', logy=True)

## Join JHU and BMP data, plot and write to file

In [ ]:
df = cases_JHU.join(cases_BMP)
df.tail(3)

In [ ]:
#df.plot(style='o-', logy=True, alpha=0.5)

In [ ]:
df.to_csv('cases.csv')